# Chapter 3 - Text Representation

![](../assets/images/Screenshot%20from%202023-10-04%2014-45-21.png)

Text representation được phân làm 4 loại:
- Basic vectorization approaches
- Distributed representations
- Universal language representation
- Handcrafted features



## 1. Vector space models

Tính toán độ tương đồng giữa 2 vector:

Cosine

![](../assets/images/Screenshot%20from%202023-10-04%2014-54-46.png)

Ngoài ra còn dùng khoảng cách Euclid

## 2. Basic Vectorization Approaches

Bắt đầu với ý tưởng đơn giản về text representation: map mỗi từ trong vocabulary (V) của text corpus tới một unique ID (integer value), 
Khi đó mỗi câu sẽ được biểu diễn bằng một vector `V` chiều.

Ví dụ có 4 văn bản:

![](../assets/images/Screenshot%20from%202023-10-04%2015-00-27.png)

Sau khi apply các phương pháp pre-processing vào ta được vocabulary: [dogs, bites, man, eats, meat, food] -> 6 từ. Như vậy với mỗi Documents bây giờ ta có thể represented bằng một vector có số chiều là 6

### 2.1 One-Hot Encoding

Ví dụ đối với vocab [dogs, bites, man, eats, meat, food]:

- dogs  -> [1, 0, 0, 0, 0, 0]
- bites -> [0, 1, 0, 0, 0, 0]
- man   -> [0, 0, 1, 0, 0, 0]
- eats  -> [0, 0, 0, 1, 0, 0]
- meat  -> [0, 0, 0, 0, 1, 0]
- food  -> [0, 0, 0, 0, 0, 1]


Như vậy với câu "Dog bites man" được biểu diễn:
[
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0]
]

Với kiểu mã hóa như thế này thì có lợi là nó khá dễ hiểu và dễ dàng thực hiện tuy nhiên gặp một số mặt hại như sau:

- Dữ liệu biểu diễn có quá nhiều giá trị 0 trong điều kiện thực thế với kích thước vocab lớn
- Không có fix length cho mỗi câu do các câu có số lượng từ khác nhau. Ta cần feature có length như nhau
- Out of vocabulary (OOV) khi triển khai model gặp phải câu có từ không có trong từ điển -> Phải train lại

### 2.2 Bag of Words

Viết tắt là BoW, là một kĩ thuật text representation cổ điển được dùng khá phổ biến trong NLP đặc biệt trong text classification problems. Với cách biểu diễn này, ta bỏ qua mối quan hệ về thứ tự và ngữ cảnh trong câu.

Giống như One-Hot Encoding về việc dùng Vocabulary vector

Ví dụ với câu "Dog bites man" được biểu diễn là [1, 1, 1, 0, 0, 0]. Hiểu như sau với vector vocab: [dogs, bites, man, eats, meat, food]. Mỗi câu sẽ được biểu diễn bởi vector 6 chiều. Trong đó giá trị của mỗi phần từ là số lần xúât hiện của từ với index tương ứng trong vocab.

In [1]:
documents = ["Dog bites man.", "Man bites dog.", "Dog eats meat.", "Man eats food."] #Same as the earlier notebook
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

#look at the documents list
print("Our corpus: ", processed_docs)

count_vect = CountVectorizer()
#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

#see the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())

#Get the representation using this vocabulary, for a new text
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

Our corpus:  ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']
Our vocabulary:  {'dog': 1, 'bites': 0, 'man': 4, 'eats': 2, 'meat': 5, 'food': 3}
BoW representation for 'dog bites man':  [[1 1 0 0 1 0]]
BoW representation for 'man bites dog:  [[1 1 0 0 1 0]]
Bow representation for 'dog and dog are friends': [[0 2 0 0 0 0]]


Ví dụ trong đoạn code trên ta thấy câu 'dog and dog are friends' có giá trị là 2 tại phần tử tương ứng với từ dog, đây là trường hợp ta quan tâm đến số lần xuát hiện của từ. Trong một số trường hợp ta không quan tâm đến việc này mà chỉ quan tâm là từ đó có xuất hiện trong câu hay không, khi đó giá trị biểu diễn của vector chỉ là 0 hoặc 1, không hoặc có xuất hiện

In [3]:
#BoW with binary vectors
count_vect = CountVectorizer(binary=True)
count_vect.fit(processed_docs)
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

Bow representation for 'dog and dog are friends': [[0 1 0 0 0 0]]


Với cách biểu diễn bằng BoW thì các câu đã được biểu diễn bởi các vector có độ dài như nhau, dễ dàng tính toán khoảng cách giữa 2 vector hơn. Tuy nhiên cách biểu diễn này có các hạn chế:

- Size của vector tăng lên khi size của vocabulary tăng. Sparsity (Có quá nhiều phần tử giá trị bằng 0) vẫn là một vấn đề. Có thể hạn chế vấn đề này bằng cách hạn chế đi số lượng vocabulary thành các từ hay gặp nhất
- Do không quan tâm về thứ tự nên 2 câu D1, D2  mặc dù là khác nghĩa nhau nhưng sẽ như nhau nếu sd BoW
- Vẫn không cải thiện đc vấn đề về từ mới xuất hiện trong thực tế không có trong từ điển

### 2.3 Bag of N-Grams

2 cách biểu diễn khảo sát ở trên treat các từ như các units độc lập, không liên quan ngữ nghĩa gì với nhau, cũng không hề quan tâm đến thứ tự. Phương pháp tiếp cận sử dụng bag of n grams (BoN) ra đời để khắc phục điều này.

Phương pháp BoN cũng break câu ra thành các phần gồm n từ liên tiếp (hay còn gọi là tokens, khác với pp trc mỗi tokens chỉ là 1 từ độc lập). Cách này giúp ta nắm bắt được 1 số ngữa cảnh. Mỗi đoạn (chunk) được gọi là `n-gram`. Như vậy corpus vocabulary, V, bây giờ là một tập hợp của các n-grams khác nhau. Sau đó mỗi document trong corpus được biểu diễn bởi một vector `V` chiều. Vector này cấu thành bởi các giá trị là số lần xuất hiện của mỗi n-gram xuất hiện trong document và giá trị 0 nếu n-gram không xuất hiện trong document.

Xét các ví dụ documents cũ ở trên ta có thể dựng được tập các 2-gram (bigrams) là {dog bites, bites man, man bites, bites dog, dog eats, eats meat, man eats, eats food}. Như vậy biểu diễn cho các documents sẽ là vector 8 chiều:

Với

D1: [1, 1, 0, 0, 0, 0, 0, 0]

D2: [0, 0, 1, 1, 0, 0, 0, 0]

Với phương pháp dùng n-gram này khi n=1 thì = BoW hay unigram, n=2 được gọi là bigram, n=3 đc gọi là trigram

Phương pháp này cũng đc gọi là `n-gram feature selection`

Ta có thể sử dụng tăng dần `n` trong `n-gram` để có thể bắt được nhiều ngữ cảnh của câu hơn.

Ví dụ

In [4]:
#our corpus
documents = ["Dog bites man.", "Man bites dog.", "Dog eats meat.", "Man eats food."]

processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

#Ngram vectorization example with count vectorizer and uni, bi, trigrams
count_vect = CountVectorizer(ngram_range=(1,3))

#Build a BOW representation for the corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Look at the vocabulary mapping
print("Our vocabulary: ", count_vect.vocabulary_)

#see the BOW rep for first 2 documents
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog: ",bow_rep[1].toarray())

#Get the representation using this vocabulary, for a new text
temp = count_vect.transform(["dog and dog are friends"])

print("Bow representation for 'dog and dog are friends':", temp.toarray())

Our vocabulary:  {'dog': 3, 'bites': 0, 'man': 12, 'dog bites': 4, 'bites man': 2, 'dog bites man': 5, 'man bites': 13, 'bites dog': 1, 'man bites dog': 14, 'eats': 8, 'meat': 17, 'dog eats': 6, 'eats meat': 10, 'dog eats meat': 7, 'food': 11, 'man eats': 15, 'eats food': 9, 'man eats food': 16}
BoW representation for 'dog bites man':  [[1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0]]
BoW representation for 'man bites dog:  [[1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0]]
Bow representation for 'dog and dog are friends': [[0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


ở ví dụ trên ta dùng ngram_range để thu được vocab là kết hợp của 1-gram, 2-gram và 3-gram

Một số mặt lợi và hạn chế của BoN:

- Bắt được ngữ cảnh và trật tự từ
- ngram càng tăng thì kích thước vector và vấn đề về sparsity tăng lên rất nhanh
- Vẫn chưa giải quyết được vấn đè OOV ( Gặp từ không có trong từ điển)

### 2.4 TF-IDF

Trong 3 cách biểu diễn đưuọc nếu trc: One-hot, BoW và BoN thì các từ trong câu được treat quan trọng như nhau, không hề có sự lưu ý, chú ý đến từ nào quan trọng hơn. Phương pháp TF-IDF (term frequency-inverse document frequency) sinh ra để giải quyết vấn đề này. TF-IDF nhằm mục đích định lượng tầm quan trọng của một từ nhất định so với các từ khác trong tài liệu và trong kho văn bản (corpus). Phương pháp này thường được sử dụng trong information-retrieval systems để tìm ra các document từ kho văn bản (corpus) với query cho bởi user.

TF-IDF thực hiện như sau: Nếu một từ `w` xuất hiện nhiều lần trong một văn bản $d_i$ nhưng hầu như không xuất hiện trong văn bản $d_j$. Vậy từ `w` sẽ rất quan trọng đối với văn bản $d_i$ nhưng lại không quan trọng lắm với $d_j$.

TF-IDF đưuọc cấu thành vởi TF và IDF

`TF` (term frequency) để đo đếm tấn suất lần xuất hiện của một từ trong một văn bản. Tuy nhiên do các văn bản có độ dài khác nhau nên có thể trong văn bản dài hơn, các từ có tấn suất xuất hiện nhiều hơn so với các văn bản ngắn hơn. Vì vậy cần phải normalize cho tổng số từ có trong văn bản. Ta có công thức


$$TF(t,d) = \frac{Number\;of\;occurrences\;of\;term\;t\;in\;document\;d}{Total\;number\;of\;terms\;in\;the\;document\;d}$$


`IDF` (inverse document frequency) đo đếm sự quan trọng của một từ, thuật ngữ trong kho văn bản. Trong việc tính toán TF, tất cả các terms được coi là quan trọng như nhau. Tuy nhiên có thể thấy rằng ví du trong tiếng anh thì các stop words như is, are, am, ... không hề quan trọng, mặc dù vậy chúng xuất hiện rất nhiều, và thường xuyên. Để giải quyết vấn đề này, IDF đánh trọng số thấp cho các terms mà rất phổ biến trong kho văn bản và đánh trọng số cao cho các terms hiếm gặp. Ta có công thức:

$$IDF(t) = log_e(\frac{Total\;number\;of\;documents\;in\;the\;corpus}{Number\;of\;documents\;with\;term\;t\;in\;them})$$

Cuối cùng TF-IDF là kết quả của tích giữa TF và IDF:

$$TF-IDF = TF * IDF$$

Ví dụ tính toán TF-IDF cho các từ trong kho văn bản của các ví dụ trước:

![](../assets/images/19.png)

Như vậy một documents sẽ được biểu diễn giống như các ví dụ trc là vector có số chiều bằng số từ trong vocabulary. Như với D1 là "Dog bites man" thì có biểu diễn như sau:

![](../assets/images/20.png)

Ví dụ với sklearn. Trong ví dụ dưới ta thấy kết quả ra có vẻ khác với công thức ở trên lý do vì sklearn sử dụng cách tính khác một chút đó là để tránh trường hợp chia cho 0 (khi số lần xuất hiện của từ trong tất cả các docs = 0) đồng thời tránh idf = 0 nên công thức của họ là:

$$idf(t) = \log(\frac{1 + n}{1 + df(t)}) + 1$$

Trong đó:

- n là số documents
- df(t) là số lần xuất hiện của từ t trong tất cả các documents
- log được hiểu là logarit cơ số tự nhiên hay $log_e$

In [6]:
documents = ["Dog bites man.", "Man bites dog.", "Dog eats meat.", "Man eats food."]
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(processed_docs)

#IDF for all words in the vocabulary
print("IDF for all words in the vocabulary",tfidf.idf_)
print("-"*10)
#All words in the vocabulary.
print("All words in the vocabulary",tfidf.get_feature_names_out())
print("-"*10)

#TFIDF representation for all documents in our corpus 
print("TFIDF representation for all documents in our corpus\n",bow_rep_tfidf.toarray()) 
print("-"*10)

temp = tfidf.transform(["dog and man are friends"])
print("Tfidf representation for 'dog and man are friends':\n", temp.toarray())

IDF for all words in the vocabulary [1.51082562 1.22314355 1.51082562 1.91629073 1.22314355 1.91629073]
----------
All words in the vocabulary ['bites' 'dog' 'eats' 'food' 'man' 'meat']
----------
TFIDF representation for all documents in our corpus
 [[0.65782931 0.53256952 0.         0.         0.53256952 0.        ]
 [0.65782931 0.53256952 0.         0.         0.53256952 0.        ]
 [0.         0.44809973 0.55349232 0.         0.         0.70203482]
 [0.         0.         0.55349232 0.70203482 0.44809973 0.        ]]
----------
Tfidf representation for 'dog and man are friends':
 [[0.         0.70710678 0.         0.         0.70710678 0.        ]]


OK tổng hợp lại tất cả các phương pháp represent text đã nêu trên ta thấy:

- Chúng đều là biểu diễn rời rạc, coi (các từ, n-grams) là units. Việc biểu diễn rời rạc như vậy làm cản trở khả năng nắm bắt mối quan hệ tương quan giữa các từ.
- Việc biểu diễn như thế này có thể gặp trường hợp bị nhiều giá trị 0 trong vector representations
- Không handle được OOV words

## 3. Distributed Representations

Phương pháp này sinh ra để khắc phục các hạn chế của những phương pháp represent bằng vector đã nêu. Dùng neural network để tạo ra dense, low-dimensional representations của các từ. Ta có một số key terms cần hiểu sau:

`Distributional similarity`

- ý tưởng này có thể được hiểu là bây giờ nghĩa của các từ không chỉ được hiểu một cách độc lập nữa mà sẽ được hiểu theo ngữ cảnh của nó. Ví dụ trong cụm "NLP rocks". Nghĩa hiểu độc lập của từ "rocks" là hòn đá tuy nhiên đặt trong ngữa cảnh của cụm từ trên thì nó được hiểu là một thứ gì đó tốt và xịn xò.

`Distributional hypothesis`

- Về mặt ngữ nghĩa, giả thuyết này nói rằng các từ được dùng trong ngữ cảnh giống nhau có ý nghĩa giống nhau. Ví dụ 2 từ "dog" và "cat" dùng trong ngữ cảnh giống nhau thì theo giả thuyết của ta thì chúng sẽ có sự tương đồng lớn về mặt ý nghĩa. Xem xét cách biểu diễn bằng vector (VSM) thì nghĩa của từ biểu diễn bằng vector vậy nếu 2 từ sử dụng trong ngữ cảnh giống nhau thì biểu diễn vector của chúng có khoảng cách gần nhau.

`Distributional representation`

- Dựa trên Distributional hypothesis, kiểu biểu diễn dựa trên số lượng từ đc dùng trong các ngữ cảnh mà nó được dùng. Nói ngắn gọn như ví dụ các cách biểu diễn vector space one-hot, BoW, BoN hay TF-IDF chính là cách biểu diễn này

`Distributed representation`

- Vẫn dựa trên Distributional hypothesis. Ta thấy rằng kiểu biểu diễn Distributional representation bị hạn chế về high dimensional và sparse. Để khắc phục vấn đề đó thì distributed representation nén chiều dữ liệu lại.


Có thể có chút confused giữa `distributional representation và distributed representation` do cách viết của chúng, nhưng chungs là 2 cách biểu diễn khác nhau. Có thể hiểu đơn giản...

`Embedding`

- Với tập các từ trong kho văn bản. Có thể hiểu embedding là sự mapping từ vector space kiểu distributional representations thành vector space representations kiểu distributed representation

`Vector semantics`

- Liên quan đến các NLP methods dùng để học cách biểu diễn từ dựa trên distributional properties của từ trong kho văn bản lớn.

### 3.1 Word embeddings

#### 3.1.1 Pre-trained word embeddings

Một số mô hình pretrained embeddings là:
- `Word2vec` của Google
- `GloVe` của Stanford
- `fasttext embeddings` của Facebook

Dimension của chúng dao động tầm 25, 50, 100, 200, 300, 600

In [8]:
import os
import wget
import gzip
import shutil

gn_vec_path = "GoogleNews-vectors-negative300.bin"

# # wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")
# gn_vec_zip_path = "GoogleNews-vectors-negative300.bin.gz"

# with gzip.open(gn_vec_zip_path, 'rb') as f_in:
#     with open(gn_vec_path, 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)
        
    

In [9]:
import warnings
warnings.filterwarnings("ignore")

import psutil
process = psutil.Process(os.getpid())
from psutil import virtual_memory
mem = virtual_memory()

import time

In [10]:
from gensim.models import Word2Vec, KeyedVectors

pretrainedpath = gn_vec_path

#Load W2V model. This will take some time, but it is a one time effort! 
pre = process.memory_info().rss
print("Memory used in GB before Loading the Model: %0.2f"%float(pre/(10**9))) #Check memory usage before loading the model
print('-'*10)

start_time = time.time() #Start the timer
ttl = mem.total #Toal memory available

w2v_model = KeyedVectors.load_word2vec_format(pretrainedpath, binary=True) #load the model
print("%0.2f seconds taken to load"%float(time.time() - start_time)) #Calculate the total time elapsed since starting the timer
print('-'*10)

print('Finished loading Word2Vec')
print('-'*10)

post = process.memory_info().rss
print("Memory used in GB after Loading the Model: {:.2f}".format(float(post/(10**9)))) #Calculate the memory used after loading the model
print('-'*10)

print("Percentage increase in memory usage: {:.2f}% ".format(float((post/pre)*100))) #Percentage increase in memory after loading the model
print('-'*10)

print("Numver of words in vocablulary: ",len(w2v_model.vocab)) #Number of words in the vocabulary. 

Memory used in GB before Loading the Model: 0.14
----------


88.05 seconds taken to load
----------
Finished loading Word2Vec
----------
Memory used in GB after Loading the Model: 4.83
----------
Percentage increase in memory usage: 3529.79% 
----------
Numver of words in vocablulary:  3000000


In [11]:
#Let us examine the model by knowing what the most similar words are, for a given word!
w2v_model.most_similar('beautiful')

[('gorgeous', 0.8353004455566406),
 ('lovely', 0.810693621635437),
 ('stunningly_beautiful', 0.7329413890838623),
 ('breathtakingly_beautiful', 0.7231341004371643),
 ('wonderful', 0.6854087114334106),
 ('fabulous', 0.6700063943862915),
 ('loveliest', 0.6612576246261597),
 ('prettiest', 0.6595001816749573),
 ('beatiful', 0.6593326330184937),
 ('magnificent', 0.6591402292251587)]

In [12]:
#What is the vector representation for a word? 
w2v_model['computer']


array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [13]:
w2v_model['compute'].shape

(300,)

In [14]:
try:
    #What if I am looking for a word that is not in this vocabulary?
    print(w2v_model['practicalnlp'])
except Exception as e:
    print(str(e))

"word 'practicalnlp' not in vocabulary"


Cần lưu ý khi sử dụng pre-trained models:

- Token/Words phải được lowercased, nếu không cũng sẽ bị word not found in vocabulary.
- Không phải tất cả các từ đều có trong pre-trained models

Sử dụng spacy để xem ví dụ lấy embedding representations cho full text

In [15]:
!python -m spacy download en_core_web_md

     ---------------------------------------- 42.8/42.8 MB 7.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [23]:
import spacy

%time 

nlp = spacy.load('en_core_web_md')
# process a sentence using the model
mydoc = nlp("sơn")
#Get a vector for individual words
#print(doc[0].vector) #vector for 'Canada', the first word in the text 
print(mydoc[0].vector) #Averaged vector for the entire sentence

Wall time: 0 ns


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### 3.1.2 Training our own embeddings

- Continuous bag of words (CBOW)
- SkipGram

Đây là hai phương pháp tìm embedding cho từ với cách thực hiện đối lập nhau. Nhưng đều chung ý tưởng là dùng mạng neural network cơ bản 1.

`CBOW`

Đây là phương pháp tìm từ trung tâm dựa vào ngữ cảnh. Với mỗi từ trong câu được chọn làm từ trung tâm, từ ngữ cảnh là các từ trong câu có vị trí cách từ đích một khoảng không quá C/2 với C là một số tự nhiên dương. Như vậy với mỗi từ trung tâm, ta sẽ có một bộ không quá C từ ngữ cảnh.

Ta sẽ tạo bộ dữ liệu training tên toàn bộ các câu của documents thuộc corpus. Mỗi sentence sẽ được dùng lấy dữ liệu như sau

Ví dụ với C = 2 

![](../assets/images/Screenshot%20from%202023-10-05%2013-53-39.png)

Như vậy với các training samples thì context là input và target là output cho mạng neural network

Mạng CBOW biểu diễn như sau:

![](../assets/images/Screenshot%20from%202023-10-05%2013-59-20.png)

Khi đó Input là vector các từ context biểu diễn dưới dạng one-hot với số chiều bằng với số chiều của vocabulary và output là từ center có biểu diễn tương tự. Kết quả cho embedding của từ center sẽ là output của hidden layer

`SkipGram`

Ngược lại với `CBOW`, SkipGram là model tìm các từ context từ center word. Cách lấy dữ liệu:

![](../assets/images/Screenshot%20from%202023-10-05%2014-03-11.png)

Kiến trúc mô hình

![](../assets/images/Screenshot%20from%202023-10-05%2014-04-41.png)

Implement Skip-gram from scratch

### 3.1 Going beyond words

Với các cách biểu diễn từ đã biết đều dùng cho từng từ đơn giản, vậy để biểu diễn cả câu thì làm như thế nào?

Phương pháp đơn giản nhất là biểu diễn từng từ, sau đó kết hợp chúng bằng cách tính tổng, hay tính trung bình cộng. Phương pháp này thì có vẻ như không bắt được hết các thuộc tính của câu như thứ tự từ tuy nhiên trong thực tế thì nó việc sử dụng như vậy rất ok.

Một vấn đề gặp phải với các phương pháp biểu diễn kể cả khi dùng word2vec, cả pre-trained hay self-trained thì đều dựa vào vocabulary có được trong quá trình training dữ liệu. Tuy nhiên khi chạy ứng dụng thực tế thì sẽ gặp phải nhiều trường hợp từ không nằm trong vocabulary hay OOV. Một phương pháp đơn giản để giải quyết vấn đề này đó là kệ các từ không có trong từ điển trong quá trình feature extraction.

Một cách khác để giải quyết vấn đề về OOV là tạo vector represent cho các từ không có trong vocab một cách ngẫu nhiên với các giá trị nằm trong đoạn từ -0.25 đến 0.25. Việc này có thể giúp cải thiện hiệu suất của model tầm 1-2%

Một cách khác để khắc phục OOV là như fastText của Facebook. Models sẽ break các từ ra thành các n-grams và biểu diễn embedding cho các n-grams này. KHi đó một từ sẽ có embedding là kết hợp của embedding của các n-grams của nó. Ví dụ với từ "gregarious" không nằm trong vocab, tuy nhiên với cách làm của fastText thì nó được break ra thành 'gre', 'reg', 'ega, ...'ous', Nên cuối cùng vẫn có present của gregarious nhờ sự kết hợp các embeddign của n-gram của nó

## 3. Distributed Representations Beyond Words and Characters

Với các phương pháp dùng Word2vect vấn gặp phải vấn đề ngữ cảnh của toàn bộ câu - trật tự từ trong câu. Ví dụ: "dog bites man" và "man bites dog". Hai câu này nếu như biểu diễn sẽ y hệt nhau nhưng lại khác nhau về mặt ngữ nghĩa.

Để giải quyết vấn đề nêu trên ta có kiểu biểu diễn khác nữa đó là Doc2Vect, lúc này ta sẽ không học từ các từ nữa mà học từ các cụm từ, các câu, đoạn văn hay toàn bộ văn bản.

Doc2Vec dựa trên paragraph vectors frame